In [2]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello, world! what is the capital of turkey?"}]
)

print(response.choices[0].message.content)

Hello! The capital of Turkey is Ankara.


In [3]:
print("helloooooow wwoolllld")

helloooooow wwoolllld
